In [1]:
train_path = "./new_train/new_train"
test_path = "./new_val_in/new_val_in"
submission_path = "./sample_submission.csv"
submission_dir = "./submissions"

In [33]:
# Train Val Split
TRAIN_SIZE = 0.85
VAL_SIZE = 0.15

# training config
NUM_EPOCH = 20
BATCH_SIZE = 64
LEARNING_RATE = 1e-3
EARLY_STOP_MAX = 6

# feature engineering configs
NEARBY_DISTANCE_THRESHOLD = 50.0  # Distance threshold to call a track as neighbor
DEFAULT_MIN_DIST_FRONT_AND_BACK = 100. # default distance 

In [34]:
import os
import copy
import pickle
import numpy as np
from glob import glob
import pandas as pd
from tqdm import tqdm
from typing import Tuple
import matplotlib.pyplot as plt


import optuna
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [35]:
from argoverse_forecasting.utils.social_features_utils import SocialFeaturesUtils

# Data 

In [36]:
class ArgoverseDataset(Dataset):
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
        
        if self.transform:
            data = self.transform(data)

        return data

In [37]:
train = ArgoverseDataset(data_path=train_path)
test = ArgoverseDataset(data_path=test_path)

In [38]:
train_size = int(TRAIN_SIZE * len(train))
val_size = len(train) - train_size
train, val = torch.utils.data.random_split(train, [train_size, val_size])

## Preprocessing

In [8]:
def get_agent_track(scene, mode="train"):
    agent_idx = np.where(scene["agent_id"] == np.unique(scene["track_id"].flatten()))[0][0]
    agent_traj = scene['p_in'][agent_idx]
    if mode == "test":
        return agent_traj
    else:
        agent_traj = np.concatenate([agent_traj, scene['p_out'][agent_idx]])
    return agent_traj

def get_social_tracks(scene, mode="train"):
    agent_idx = np.where(scene["agent_id"] == np.unique(scene["track_id"].flatten()))[0][0]
    social_masks = scene["car_mask"].flatten()
    social_masks[agent_idx] = 0
    social_trajs = scene['p_in'][social_masks.astype(bool)]
    
    if mode == "test":
        return social_trajs
    else:
        return np.concatenate([social_trajs, scene['p_out'][social_masks.astype(bool)]], axis=1)

In [24]:
# generate social features
def get_social_features(scene, mode="train"):
    """
    Extract social features:
        1. number of neighbors
        2. min front/back distance at each timestamp
    """
    agent_track = get_agent_track(scene, mode)
    social_tracks = get_social_tracks(scene, mode)
    # compute social features
    if mode == "test":
        num_neighbors = count_num_neighbors(agent_track, social_tracks)
    else:
        num_neighbors = count_num_neighbors(agent_track[:19], social_tracks[:, :19])
#     min_dist = get_min_distance_front_and_back(agent_traj, social_trajs)
#     return np.concatenate((num_neighbors, min_dist), axis=1)
#     social_features_utils = SocialFeaturesUtils()
#     min_dist_front_back = social_features_utils.get_min_distance_front_and_back(
#         agent_track=agent_track,
#         social_tracks=social_tracks,
#         obs_len=19,
#         raw_data_format={"X": 0, "Y": 1}
#     )
    
#     return np.concatenate((num_neighbors, min_dist_front_back), axis=1)
    return num_neighbors

In [10]:
def count_num_neighbors(agent_traj, social_trajs):
    """
    Calculate euclidean distance between agent_traj and social_trajs
    if distance is less than NEARBY_DISTANCE_THRESHOLD, then num_neighbors++
    
    Args:
        agent_traj (np.array): data for agent trajectory
        social_trajs (np.array): array of other agents' trajectories
    Returns:
        (np.array): 
    """
    num_neighbors = []
    dist = np.sqrt(
        (social_trajs[:, :, 0] - agent_traj[:, 0])**2 
        + (social_trajs[:, :, 1] - agent_traj[:, 1])**2
    ).T
    num_neighbors = np.sum(dist < NEARBY_DISTANCE_THRESHOLD, axis=1)
    return num_neighbors.reshape(-1, 1)

In [40]:
def my_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp, out = [], []
    for scene in batch:
        agent_idx = np.where(scene["agent_id"] == np.unique(scene["track_id"].flatten()))[0][0]
        social_features = get_social_features(scene)
        inp.append(np.hstack([scene['p_in'][agent_idx], social_features])) # scene['v_in'][agent_idx],
        out.append(scene['p_out'][agent_idx])

    inp = torch.FloatTensor(inp)
    out = torch.FloatTensor(out)
    return [inp, out]

def my_test_collect(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp, out = [], []
    for scene in batch:
        agent_idx = np.where(scene["agent_id"] == np.unique(scene["track_id"].flatten()))[0][0]
        social_features = get_social_features(scene, mode="test")
        inp.append(np.hstack([scene['p_in'][agent_idx], social_features])) #  scene['v_in'][agent_idx],
        out.append([])

    inp = torch.FloatTensor(inp)
    out = torch.FloatTensor(out)
    return [inp, out]


train_loader = DataLoader(train, batch_size=BATCH_SIZE, shuffle = False, collate_fn=my_collate, num_workers=0)
val_loader = DataLoader(val, batch_size=BATCH_SIZE, shuffle = False, collate_fn=my_collate, num_workers=0)
test_loader = DataLoader(test, batch_size=BATCH_SIZE, shuffle = False, collate_fn=my_test_collect, num_workers=0)
# exmaple = iter(val_loader)
# exmaple.next()[0][0]

In [12]:
submission = pd.read_csv(submission_path)
submission

,ID,v1,v2,v3,v4,v5,v6,v7,v8,v9,...,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60
0,10002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10028,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,9897,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3196,99,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3197,9905,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3198,9910,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Social features

# Data preprocessing

In [13]:
def get_relative_position(inp, out=None):
    """calculate position difference"""
    input_length = inp.shape[1]
    p_in_0 = copy.deepcopy(inp[:, 0, :2])
    
    
    if out is not None:
        out = copy.deepcopy(out)
        output_length = out.shape[1]
        for i in range(output_length - 1, 0, -1):
            out[:, i, :2] = out[:, i, :2] - out[:, i - 1, :2]
        out[:, 0, :2] = out[:, 0, :2] - inp[:, -1, :2]
        
    for i in range(input_length - 1, 0, -1):
        inp[:, i, :2] = inp[:, i, :2] - inp[:, i - 1, :2]
    inp[:, 0, :] = 0
    
    if out is not None:
        return inp, out, p_in_0
    
    return inp, p_in_0

In [14]:
def get_absolute_position(inp, out, p_in_0, return_pred_only=False):
    """position inverse difference"""
    inp[:, 0, :2] = p_in_0
    for i in range(1, inp.shape[1]):
        inp[:, i, :2] = inp[:, i, :2] + inp[:, i - 1, :2]

    out[:, 0, :2] = out[:, 0, :2] + inp[:, -1, :2]
    for i in range(1, out.shape[1]):
        out[:, i, :2] = out[:, i, :2] + out[:, i - 1, :2]
    
    if return_pred_only:
        return out
    
    return inp, out

# Training

In [15]:
# model training
is_cuda = torch.cuda.is_available()
if is_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
device

device(type='cuda')

In [16]:
class RMSELoss(torch.nn.Module):
    """RMSE Loss"""
    def __init__(self):
        super(RMSELoss, self).__init__()

    def forward(self, yhat, y):
        criterion = nn.MSELoss()
        loss = torch.sqrt(criterion(yhat, y))
        return loss

## Seq2Seq (LSTM)

In [17]:
# training config
ROLLOUT_LEN = 30

# model config
INPUT_SIZE = 5
EMBEDDING_SIZE = 8
HIDDEN_SIZE = 16
OUTPUT_SIZE = 2

In [18]:
class EncoderLSTM(nn.Module):
    def __init__(self, 
                 input_size,
                 embedding_size=EMBEDDING_SIZE, 
                 num_layers=3, 
                 hidden_size=HIDDEN_SIZE):
        super(EncoderLSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.linear = nn.Linear(input_size, embedding_size)
        self.lstms = nn.ModuleList([nn.LSTMCell(
            embedding_size if i == 0 else hidden_size, hidden_size
        ) for i in range(self.num_layers)])
           
#         self.lstm = nn.LSTM(embedding_size, hidden_size, num_layers=self.num_layers)

    def init_hidden(self, batch_size, hidden_size, num_layers):
        """Return a list of initial hidden states"""
        # Initialize encoder hidden state
        hiddens = []
        for i in range(num_layers):
            h_0 = torch.zeros(batch_size, hidden_size).to(device)
            c_0 = torch.zeros(batch_size, hidden_size).to(device)
            nn.init.xavier_normal_(h_0)
            nn.init.xavier_normal_(c_0)
            hiddens.append((h_0, c_0))
        return hiddens
    
    def forward(self, X, last_hidden_states=None):
        if last_hidden_states is None:
            last_hidden_states = self.init_hidden(X.shape[0], self.hidden_size, self.num_layers)
        
        # original: (batch_size, seq_len, input_size)
        # expected: (seq_len, batch_size, input_size)
        embedded = F.relu(self.linear(X))
        hiddent_states = []
        for i in range(self.num_layers):
            hidden_state = self.lstms[i](embedded if i == 0 else hidden_state[0], last_hidden_states[i])
            hiddent_states.append(hidden_state)
        return hiddent_states

In [19]:
class DecoderLSTM(nn.Module):
    def __init__(
        self, 
        embedding_size=EMBEDDING_SIZE,
        hidden_size=HIDDEN_SIZE, 
        num_layers=3,
        output_size=OUTPUT_SIZE
    ):
        super(DecoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.linear1 = nn.Linear(output_size, embedding_size)
        self.lstms = nn.ModuleList([nn.LSTMCell(
            embedding_size if i == 0 else hidden_size, hidden_size
        ) for i in range(self.num_layers)])
        self.dropout = nn.Dropout(p=0.2)
        self.linear2 = nn.Linear(hidden_size, output_size)

    def forward(self, decoder_input, last_encoder_hiddens):
        embedded = self.dropout(F.relu(self.linear1(decoder_input)))
        hidden_states = []
        for i, lstm in enumerate(self.lstms):
            hidden = lstm(embedded if i == 0 else hidden[0], last_encoder_hiddens[i])
            hidden_states.append(hidden)
        output = self.linear2(hidden[0])
        return output, hidden_states

In [44]:
def check_validation(val_loader, encoder, decoder, normalize=False):
    loss_fn = RMSELoss()
    
    for i_batch, batch_data in enumerate(val_loader):
        inp, out = batch_data
        inp = inp.to(device)
        out = out.to(device)
        
        # eval mode
        encoder.eval()
        decoder.eval()
        
        # Encoder
        batch_size = inp.shape[0]
        input_length = inp.shape[1]
        output_length = out.shape[1]
        input_shape = inp.shape[2]

        # Initialize losses
        loss = 0

        # Get relative position
        initial_p_in = inp[:, 0, :2].detach().clone()
        inp[:, :, :2] = inp[:, :, :2] - initial_p_in[:, None]
#         inp, processed_out, p_in_0 = get_relative_position(inp, out)

        # Encode observed trajectory
        encoder_hidden = None
        for i in range(input_length):
            encoder_input = inp[:, i, :]
            encoder_hidden = encoder(encoder_input, encoder_hidden)

        # Initialize decoder input with last coordinate in encoder
        decoder_hidden = encoder_hidden
        decoder_input = encoder_input[:, :2]

        # Decode hidden state in future trajectory
#         prev_p = p_in_0
        for i in range(output_length):
            decoder_output, decoder_hidden = decoder(decoder_input, encoder_hidden)

            # Update loss
            loss += loss_fn(decoder_output[:, :2], out[:, i, :2] - initial_p_in)
#             pred_out = decoder_output[:, :2] + prev_p
#             loss += loss_fn(pred_out, out[:, i, :2])
#             prev_p = pred_out

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        loss = loss / output_length
    return loss

In [45]:
# model training
encoder = EncoderLSTM(3)
decoder = DecoderLSTM()
encoder.to(device)
decoder.to(device)

loss_fn = RMSELoss()
encoder_optimizer = torch.optim.Adam(encoder.parameters(), lr=LEARNING_RATE)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=LEARNING_RATE)

In [46]:
# training
train_losses = []
val_losses = []
prev_loss, non_decreasing_loss_cnt = -float("inf"), 0
for epoch in range(NUM_EPOCH):
    for i_batch, batch_data in enumerate(train_loader):
        inp, out = batch_data
        inp = inp.to(device)
        out = out.to(device)
        
        # Set to train mode
        encoder.train()
        decoder.train()

        # Initialize losses
        loss = 0

        # Encoder
        batch_size = inp.shape[0]
        input_length = inp.shape[1]  # expected: 19
        output_length = out.shape[1]  # expected: 30
        input_shape = inp.shape[2]
        total_length = input_length + output_length  # expected: 49

        # Get relative position  
        initial_p_in = inp[:, 0, :2].detach().clone()
        inp[:, :, :2] = inp[:, :, :2] - initial_p_in[:, None]
        
#         inp, processed_out, p_in_0 = get_relative_position(inp, out)
#         inp_cols = [0, 1, 4]
        # Encode observed trajectory
        encoder_hidden = None
        for i in range(input_length):
            encoder_input = inp[:, i, :]
            encoder_hidden = encoder(encoder_input, encoder_hidden)
#         encoder_hidden = encoder(inp)
        
        # Initialize decoder input with last coordinate in encoder
        decoder_hidden = encoder_hidden
        decoder_input = encoder_input[:, :2]

        # Decode hidden state in future trajectory
#         prev_p = p_in_0
        for i in range(output_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)

            # Update loss
            loss += loss_fn(decoder_output[:, :2], out[:, i, :2] - initial_p_in)
            
#             pred_out = decoder_output[:, :2] + prev_p
#             loss += loss_fn(decoder_output[:, :2], processed_out[:, i, :2])
#             prev_p = pred_out

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        # Get average loss for pred_len
        loss = loss / output_length

        # Backpropagate
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        
            
        if i_batch % 400 == 0:
            # print training loss
            training_loss = loss.item()
            train_losses.append(training_loss)
        
            # validate 
            with torch.no_grad():
                val_loss = check_validation(val_loader, encoder, decoder, normalize=False)
#                 val_losses.append(val_loss.item())  
            
            print(f"Epoch {epoch+1}/{NUM_EPOCH}, batch {i_batch}/{len(train_loader)}, "\
                    + f"Training loss: {training_loss:.4f}"\
                    + f", Val loss: {val_loss.item():.4f}")
            
            # early stop
#             if val_loss.item() > prev_loss:
#                 non_decreasing_loss_cnt += 1
#             else:
#                 non_decreasing_loss_cnt = 0
#                 prev_loss = val_loss.item()
            
#             if non_decreasing_loss_cnt >= EARLY_STOP_MAX:
#                 break 
                
#     if non_decreasing_loss_cnt >= EARLY_STOP_MAX:
#         break 

Epoch 1/20, batch 0/2736, Training loss: 23.8862, Val loss: 23.1506
Epoch 1/20, batch 400/2736, Training loss: 17.6191, Val loss: 17.8455
Epoch 1/20, batch 800/2736, Training loss: 15.8906, Val loss: 16.2224
Epoch 1/20, batch 1200/2736, Training loss: 14.5496, Val loss: 15.9050
Epoch 1/20, batch 1600/2736, Training loss: 11.1741, Val loss: 14.6491
Epoch 1/20, batch 2000/2736, Training loss: 10.7174, Val loss: 13.3682
Epoch 1/20, batch 2400/2736, Training loss: 7.1087, Val loss: 12.1975
Epoch 2/20, batch 0/2736, Training loss: 7.8157, Val loss: 11.8589
Epoch 2/20, batch 400/2736, Training loss: 6.1122, Val loss: 11.4944
Epoch 2/20, batch 800/2736, Training loss: 5.0937, Val loss: 11.2545
Epoch 2/20, batch 1200/2736, Training loss: 4.7789, Val loss: 11.2387
Epoch 2/20, batch 1600/2736, Training loss: 4.4533, Val loss: 11.1336
Epoch 2/20, batch 2000/2736, Training loss: 4.4217, Val loss: 10.8512
Epoch 2/20, batch 2400/2736, Training loss: 3.3292, Val loss: 10.7126
Epoch 3/20, batch 0/2736

KeyboardInterrupt: 

In [ ]:
sum(p.numel() for p in decoder.parameters())

In [27]:
# predict
pred_p_out = []
all_zeros_ix = []
with torch.no_grad():
    for i_batch, batch_data in enumerate(test_loader):
        inp, out = batch_data
        inp = inp.to(device)
                    
        # Set to eval mode
        encoder.eval()
        decoder.eval()

        # Encoder
        batch_size = inp.shape[0]
        input_length = inp.shape[1]
        input_shape = inp.shape[2]

        # Get relative position
        initial_p_in = inp[:, 0, :2].detach().clone()
        inp[:, :, :2] = inp[:, :, :2] - initial_p_in[:, None]
#         inp, p_in_0 = get_relative_position(inp)
        
        # Encode observed trajectory
        for i in range(input_length):
            encoder_input = inp[:, i, :]
            encoder_hidden = encoder(encoder_input)

        # Initialize decoder input with last coordinate in encoder
        decoder_hidden = encoder_hidden
        decoder_input = encoder_input[:, :2]
        decoder_outputs = torch.zeros((len(inp), 30, 2))

        # Decode hidden state in future trajectory
        for i in range(30):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            decoder_outputs[:, i, :] = decoder_output + initial_p_in

            # Use own predictions as inputs at next step
            decoder_input = decoder_output
            
        # predicted_p_out
#         out = get_absolute_position(inp, decoder_outputs.to(device), p_in_0, return_pred_only=True)
        pred_p_out.append(decoder_outputs)

In [28]:
predicted_p_out = torch.FloatTensor(len(test_loader), 30, 2)
torch.cat(pred_p_out, out=predicted_p_out)

tensor([[[1718.1674,  349.4510],
         [1721.1884,  355.0500],
         [1723.6831,  359.9641],
         ...,
         [1704.9786,  420.0189],
         [1704.4425,  420.3767],
         [1704.0544,  420.6542]],

        [[ 728.6434, 1227.5297],
         [ 726.5579, 1223.2423],
         [ 724.8978, 1219.6112],
         ...,
         [ 712.0584, 1162.3185],
         [ 711.7137, 1161.3962],
         [ 711.4163, 1160.6118]],

        [[ 585.1850, 1249.8937],
         [ 586.6434, 1251.0968],
         [ 587.8399, 1252.3562],
         ...,
         [ 610.0473, 1296.7715],
         [ 610.4717, 1298.2325],
         [ 610.8462, 1299.6322]],

        ...,

        [[1757.1272,  444.5212],
         [1756.6501,  447.2381],
         [1756.0437,  449.5727],
         ...,
         [1730.6949,  472.9033],
         [1729.5703,  473.2854],
         [1728.4368,  473.6598]],

        [[ 583.9580, 1288.6854],
         [ 583.2294, 1287.1013],
         [ 582.3897, 1285.6603],
         ...,
         [ 570.76

# Submission

In [29]:
# make final submission dataframe
df = pd.DataFrame(predicted_p_out.view(3200, -1).cpu().numpy(), columns=[f"v{i}" for i in range(1, 61)])
df.head()

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60
0,1718.167358,349.451019,1721.188354,355.049988,1723.683105,359.964111,1725.946655,364.665222,1728.003052,369.199066,...,1706.635620,418.986298,1705.698486,419.561737,1704.978638,420.018860,1704.442505,420.376678,1704.054443,420.654175
1,728.643372,1227.529663,726.557922,1223.242310,724.897827,1219.611206,723.590637,1216.297607,722.560852,1213.222290,...,712.900696,1164.670898,712.451050,1163.398804,712.058411,1162.318481,711.713684,1161.396240,711.416321,1160.611816
2,585.184998,1249.893677,586.643433,1251.096802,587.839905,1252.356201,588.974243,1253.730713,590.088745,1255.198364,...,609.061584,1293.652100,609.576538,1295.245361,610.047302,1296.771484,610.471680,1298.232544,610.846191,1299.632202
3,1711.302490,335.212433,1717.384888,341.419952,1722.428955,347.357880,1726.825317,353.189972,1730.555908,358.519531,...,1742.168213,386.077820,1741.931152,386.440338,1741.685059,386.775360,1741.431641,387.085114,1741.172485,387.371765
4,2127.204346,669.752380,2122.890869,662.372009,2120.180664,656.191040,2118.242920,650.470276,2117.015381,645.193726,...,2117.430176,625.168640,2117.488770,625.222778,2117.552979,625.284363,2117.621338,625.351379,2117.693115,625.422119


In [30]:
df['ID'] = submission['ID']
cols = df.columns[-1:].tolist() + df.columns[:-1].tolist()
df = df[cols]
df

,ID,v1,v2,v3,v4,v5,v6,v7,v8,v9,...,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60
0,10002,1718.167358,349.451019,1721.188354,355.049988,1723.683105,359.964111,1725.946655,364.665222,1728.003052,...,1706.635620,418.986298,1705.698486,419.561737,1704.978638,420.018860,1704.442505,420.376678,1704.054443,420.654175
1,10015,728.643372,1227.529663,726.557922,1223.242310,724.897827,1219.611206,723.590637,1216.297607,722.560852,...,712.900696,1164.670898,712.451050,1163.398804,712.058411,1162.318481,711.713684,1161.396240,711.416321,1160.611816
2,10019,585.184998,1249.893677,586.643433,1251.096802,587.839905,1252.356201,588.974243,1253.730713,590.088745,...,609.061584,1293.652100,609.576538,1295.245361,610.047302,1296.771484,610.471680,1298.232544,610.846191,1299.632202
3,10028,1711.302490,335.212433,1717.384888,341.419952,1722.428955,347.357880,1726.825317,353.189972,1730.555908,...,1742.168213,386.077820,1741.931152,386.440338,1741.685059,386.775360,1741.431641,387.085114,1741.172485,387.371765
4,1003,2127.204346,669.752380,2122.890869,662.372009,2120.180664,656.191040,2118.242920,650.470276,2117.015381,...,2117.430176,625.168640,2117.488770,625.222778,2117.552979,625.284363,2117.621338,625.351379,2117.693115,625.422119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,9897,255.183472,811.148438,254.884674,811.865417,254.695724,812.843689,254.620331,814.127747,254.618958,...,247.922150,888.438538,248.659195,891.403442,249.213135,893.687988,249.519669,895.287903,249.598785,896.309875
3196,99,578.807251,1136.117554,577.086121,1128.628906,576.118164,1122.508179,575.527588,1117.266113,575.108704,...,568.495239,1086.669922,568.653564,1086.731567,568.830627,1086.807007,569.023376,1086.892334,569.228943,1086.984131
3197,9905,1757.127197,444.521240,1756.650146,447.238098,1756.043701,449.572723,1755.417480,451.688660,1754.731079,...,1732.925903,472.137390,1731.812866,472.519531,1730.694946,472.903290,1729.570312,473.285431,1728.436768,473.659821
3198,9910,583.958008,1288.685425,583.229431,1287.101318,582.389709,1285.660278,581.614868,1284.301147,580.901245,...,571.434692,1262.151855,571.103333,1261.875977,570.769592,1261.672485,570.431641,1261.539551,570.087463,1261.475098


In [31]:
from datetime import datetime
def save_submission(df, filename):
    filename = filename + "_" + str(datetime.now()) + ".csv"
    file_path = os.path.join(submission_dir, filename)
    df.to_csv(file_path, index=False)

In [32]:
save_submission(df, "stacked LSTM + Social(num_neighbors) + relave_position + epoch20")